In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('clean_products') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
##extract table compras from BigQuery Staging ######

In [5]:
#name table compras
table_compras = "becade_mgutierrez.pr_compras"

#load table
raw_compras = spark.read \
  .format("bigquery") \
  .option("table", table_compras) \
  .load()

#show schema
raw_compras.printSchema()

#show incoming lines
print("lines incoming: " , raw_compras.count())

root
 |-- client_id: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_quantity: long (nullable = true)
 |-- purchase_date: date (nullable = true)

lines incoming:  1965807


In [6]:
from pyspark.sql.functions  import year, month

df_new_sales= raw_compras.withColumn('month_sales',month(raw_compras.purchase_date)) \
                .withColumn('year_sales',year(raw_compras.purchase_date))

df_new_sales.show(2)
#df_new_sales.printSchema()
print("lines source: " , raw_compras.count())

+-----------------+-------------+----------+----------------+-------------+-----------+----------+
|        client_id|product_price|product_id|product_quantity|purchase_date|month_sales|year_sales|
+-----------------+-------------+----------+----------------+-------------+-----------+----------+
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2012-03-16|          3|      2012|
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2013-03-22|          3|      2013|
+-----------------+-------------+----------+----------------+-------------+-----------+----------+
only showing top 2 rows

lines source:  1965807


In [ ]:
############ YEAR ######################

In [12]:
from pyspark.sql.functions  import countDistinct

###compras por Año
df_ordenes_year = df_new_sales.select('year_sales','purchase_date','month_sales','client_id') \
        .groupBy('year_sales','purchase_date','client_id') \
        .agg(countDistinct('client_id').alias('total_compras')) \
        .sort(['year_sales', 'purchase_date'], ascending=True)

#df_year= df_year.withColumn('venta_total', df_year.venta_total.cast(DecimalType(18, 2)))

df_ordenes_year.show(5)
# df_ordenes_year.printSchema() 

+----------+-------------+-----------------+-------------+
|year_sales|purchase_date|        client_id|total_compras|
+----------+-------------+-----------------+-------------+
|      2010|   2010-01-01|602-878245-53-323|            1|
|      2010|   2010-01-01|661-717208-27-804|            1|
|      2010|   2010-01-01|408-329226-28-494|            1|
|      2010|   2010-01-01|831-960523-53-330|            1|
|      2010|   2010-01-01|805-787952-37-812|            1|
+----------+-------------+-----------------+-------------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions  import count

sum_ordenes_year = df_ordenes_year.select('year_sales','total_compras') \
        .groupBy('year_sales') \
        .agg(count('total_compras').alias('total_compras')) \
        .sort('year_sales', ascending=True)
sum_ordenes_year.show()

+----------+-------------+
|year_sales|total_compras|
+----------+-------------+
|      2010|        32717|
|      2011|        32746|
|      2012|        32899|
|      2013|        32733|
|      2014|        32713|
|      2015|        32957|
|      2016|        32854|
|      2017|        32753|
|      2018|        32724|
|      2019|        32755|
|      2020|        32913|
+----------+-------------+



In [19]:
from pyspark.sql.functions  import sum,avg

df_sales = df_new_sales.select('year_sales','month_sales','product_price','client_id') \
        .groupBy('year_sales') \
        .agg(sum('product_price').alias('venta_total_year'), \
             avg('product_price').alias('avg_venta_mensual')) \
         .sort('year_sales', ascending=True)
            

#df_year= df_year.withColumn('venta_total', df_year.venta_total.cast(DecimalType(18, 2)))

df_sales.show()
#df_year.printSchema() 

+----------+--------------------+------------------+
|year_sales|    venta_total_year| avg_venta_mensual|
+----------+--------------------+------------------+
|      2010|1.8495069750010703E7|104.32923661415366|
|      2011|1.8595521430009063E7|104.14100184255835|
|      2012| 1.891395686000886E7|104.80852955197692|
|      2013| 1.855743942000939E7|103.84979669274121|
|      2014|1.8657688250007752E7| 105.1688391664802|
|      2015| 1.872384307000746E7|104.68142492945773|
|      2016| 1.867335246000713E7|104.62258287908166|
|      2017| 1.872034121000714E7|104.20509554746833|
|      2018|1.8488971880010866E7|104.16204820233499|
|      2019|1.8643064240007747E7|104.42012243827817|
|      2020|1.8892210430008642E7|104.74258421676042|
+----------+--------------------+------------------+



In [22]:
from pyspark.sql.functions  import col

#InnerJoin
full_table_year = df_sales.alias('A').join(sum_ordenes_year.alias('B'), col('A.year_sales') == col('B.year_sales'), "inner") 

#Show first 20 rows
full_table_year= full_table_year.select('A.year_sales','A.venta_total_year','A.avg_venta_mensual','B.total_compras') \
                 .sort('A.year_sales', ascending=True)

full_table_year.show()


+----------+--------------------+------------------+-------------+
|year_sales|    venta_total_year| avg_venta_mensual|total_compras|
+----------+--------------------+------------------+-------------+
|      2010|1.8495069750010703E7|104.32923661415366|        32717|
|      2011|1.8595521430009063E7|104.14100184255835|        32746|
|      2012| 1.891395686000886E7|104.80852955197692|        32899|
|      2013| 1.855743942000939E7|103.84979669274121|        32733|
|      2014|1.8657688250007752E7| 105.1688391664802|        32713|
|      2015| 1.872384307000746E7|104.68142492945773|        32957|
|      2016| 1.867335246000713E7|104.62258287908166|        32854|
|      2017| 1.872034121000714E7|104.20509554746833|        32753|
|      2018|1.8488971880010866E7|104.16204820233499|        32724|
|      2019|1.8643064240007747E7|104.42012243827817|        32755|
|      2020|1.8892210430008642E7|104.74258421676042|        32913|
+----------+--------------------+------------------+----------

In [23]:
########Insert table pr_compras_anuales  to BigQuery Production ###############
#################################################################################

In [24]:
full_table_year.write \
  .format("bigquery") \
  .option("table","becade_mgutierrez.pr_compras_anuales") \
  .option("temporaryGcsBucket", "amazon_magdielgutierrez") \
  .mode('overwrite') \
  .save()

In [64]:
################ MONTH ####################

In [31]:
from pyspark.sql.functions  import countDistinct
#### datafrem  df_new_sales

###compras por Mes
df_ordenes_month = df_new_sales.select('year_sales','purchase_date','month_sales','client_id') \
        .groupBy('year_sales','month_sales','purchase_date','client_id') \
        .agg(countDistinct('client_id').alias('total_compras')) \
        .sort(['year_sales', 'purchase_date'], ascending=True)

#df_year= df_year.withColumn('venta_total', df_year.venta_total.cast(DecimalType(18, 2)))

df_ordenes_month.show(5)
# df_ordenes_year.printSchema() 

+----------+-----------+-------------+-----------------+-------------+
|year_sales|month_sales|purchase_date|        client_id|total_compras|
+----------+-----------+-------------+-----------------+-------------+
|      2010|          1|   2010-01-01|213-242594-75-886|            1|
|      2010|          1|   2010-01-01|323-079732-31-237|            1|
|      2010|          1|   2010-01-01|928-662647-10-781|            1|
|      2010|          1|   2010-01-01|559-163720-78-708|            1|
|      2010|          1|   2010-01-01|480-970965-40-063|            1|
+----------+-----------+-------------+-----------------+-------------+
only showing top 5 rows



In [32]:
from pyspark.sql.functions  import count

sum_ordenes_month = df_ordenes_month.select('year_sales','month_sales','total_compras') \
        .groupBy('year_sales','month_sales') \
        .agg(count('total_compras').alias('total_compras_mes')) \
        .sort(['year_sales','month_sales'], ascending=True)

print("lines source: " , sum_ordenes_month.count())
sum_ordenes_month.show(13)

lines source:  132
+----------+-----------+-----------------+
|year_sales|month_sales|total_compras_mes|
+----------+-----------+-----------------+
|      2010|          1|             2618|
|      2010|          2|             2406|
|      2010|          3|             2478|
|      2010|          4|             2778|
|      2010|          5|             2786|
|      2010|          6|             2879|
|      2010|          7|             2958|
|      2010|          8|             2889|
|      2010|          9|             2679|
|      2010|         10|             2697|
|      2010|         11|             2580|
|      2010|         12|             2969|
|      2011|          1|             2586|
+----------+-----------+-----------------+
only showing top 13 rows



In [49]:
from pyspark.sql.functions  import sum,avg

df_month = df_new_sales.select('year_sales','month_sales','product_price') \
        .groupBy('year_sales','month_sales') \
        .agg(sum('product_price').alias('venta_total_mes')) \
        .sort(['year_sales','month_sales'], ascending=True)

#df_year= df_year.withColumn('venta_total', df_year.venta_total.cast(DecimalType(18, 2)))
print("lines source: " , df_month.count())
df_month.show(13)
#df_year.printSchema() 

lines source:  132
+----------+-----------+------------------+
|year_sales|month_sales|   venta_total_mes|
+----------+-----------+------------------+
|      2010|          1|1241494.2399999024|
|      2010|          2|1230857.7399999062|
|      2010|          3| 1118628.349999931|
|      2010|          4|  1628756.23999979|
|      2010|          5|1518457.3699998613|
|      2010|          6|1991022.5199997695|
|      2010|          7| 1960883.829999772|
|      2010|          8|1668432.4099997922|
|      2010|          9|1476454.5499998732|
|      2010|         10|1468640.0599998764|
|      2010|         11| 1271374.029999895|
|      2010|         12|1920068.4099997666|
|      2011|          1|1220903.2599999004|
+----------+-----------+------------------+
only showing top 13 rows



In [50]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

df_month_raw = df_month.withColumn('venta_total_mes_anterior',lag(df_month['venta_total_mes']).over(Window.orderBy("month_sales","year_sales")))
print("lines source: " , df_month_raw.count())


lines source:  132


In [51]:
df_month_raw= df_month_raw.na.fill(value=0,subset=["venta_total_mes_anterior"])
df_month_raw.show(5)

+----------+-----------+------------------+------------------------+
|year_sales|month_sales|   venta_total_mes|venta_total_mes_anterior|
+----------+-----------+------------------+------------------------+
|      2010|          1|1241494.2399999024|                     0.0|
|      2011|          1|1220903.2599999004|      1241494.2399999024|
|      2012|          1|1207546.9799999103|      1220903.2599999004|
|      2013|          1|1229377.7699999062|      1207546.9799999103|
|      2014|          1|1288639.0199998915|      1229377.7699999062|
+----------+-----------+------------------+------------------------+
only showing top 5 rows



In [52]:
#InnerJoin
full_table_month = df_month_raw.alias('A').join(sum_ordenes_month.alias('B'), \
                (col('A.month_sales') == col('B.month_sales')) & (col('A.year_sales') == col('B.year_sales')) , "inner") 

#Show first 20 rows
full_table_month= full_table_month.select('A.year_sales','A.month_sales','A.venta_total_mes','venta_total_mes_anterior','B.total_compras_mes') \
              .sort(['month_sales','year_sales'], ascending=True)

print("lines source: " , full_table_month.count())
full_table_month.show(13)

lines source:  132
+----------+-----------+------------------+------------------------+-----------------+
|year_sales|month_sales|   venta_total_mes|venta_total_mes_anterior|total_compras_mes|
+----------+-----------+------------------+------------------------+-----------------+
|      2010|          1|1241494.2399999024|                     0.0|             2618|
|      2011|          1|1220903.2599999004|      1241494.2399999024|             2586|
|      2012|          1|1207546.9799999103|      1220903.2599999004|             2577|
|      2013|          1|1229377.7699999062|      1207546.9799999103|             2569|
|      2014|          1|1288639.0199998915|      1229377.7699999062|             2602|
|      2015|          1|1270299.0199999036|      1288639.0199998915|             2630|
|      2016|          1|1345551.9299998868|      1270299.0199999036|             2619|
|      2017|          1|1145680.2099999334|      1345551.9299998868|             2529|
|      2018|          1|

In [53]:
full_table_month.write \
  .format("bigquery") \
  .option("table","becade_mgutierrez.pr_compras_mensuales") \
  .option("temporaryGcsBucket", "amazon_magdielgutierrez") \
  .mode('overwrite') \
  .save()